In [ ]:
# --- Cell 1 ---
%matplotlib inline
import numpy as np
import jax.numpy as jnp
from jax import jacfwd
import proxsuite

import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="vpython")

In [ ]:
# --- Cell 2 ---
N = 6 # dimension of state
M = 2 # dimension of action

t = 0.             # t0
tf = 60.           # total simulation time
dt = 0.05          # simulation time step
S = int(tf/dt) + 1 # total steps
print(f"S = {S}")

# Parameters
g = 9.81 # gravity (m/s^2)
m = 1.   # mass (m)
J = 0.05 # kg*m^2
kx = 0.3
ky = 0.3
k_theta = 0.02

# Limits of thrust (T) and torque (τ)
Tmin = 1/2 * m * g
Tmax = 3/2 * m * g
tau_min = -1/2
tau_max = 1/2

In [ ]:
# --- Cell 3 ---
# Cost matrices
Q = np.diag([1., 1., 10., 0.1, 0.1, 1])
R = 0.1 * np.eye(M)
QF = np.eye(N)

In [ ]:
# --- Cell 4 ---
def simulation(xs, x0, x_ref, traj=None):
    states = np.hstack(xs)
    
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_xlim(-101, 101)
    ax.set_ylim(-101, 101)
    ax.set_aspect('equal')
    ax.grid(True)

    plt.rcParams['animation.embed_limit'] = 100.0
    
    if traj is not None:
        all_x = np.concatenate([states[0,:], traj[:,0]])
        all_y = np.concatenate([states[1,:], traj[:,1]])
        
        title = "Trajectory Tracking"        
        ax.plot(traj[:, 0], traj[:, 1], 'r--', label='Reference Path', alpha=0.6)
    else:
        title = "Hover"
        target_point, = ax.plot([], [], 'ro', markersize=5)
        
    ax.set_title(f'Quadrotor Control via Convex MPC ({title})')    
    
    L = 3.
    quadrotor_L,  = ax.plot([], [], 'o-', lw=1, color='black')
    init_point,   = ax.plot([], [], 'bo', markersize=5)
    
    def init():
        quadrotor_L.set_data([], [])
        init_point.set_data([x0[0,0]], [x0[1,0]])
        
        if traj is None:
            target_point.set_data([x_ref[0,0]], [x_ref[1,0]])
        
        return quadrotor_L,
    
    def animate(i):
        if (i >= states.shape[1]):
            return quadrotor_L, init_point, target_point

        x = states[0, i]
        y = states[1, i]
        theta = states[2, i]

        # Left side
        xl = x - L * np.cos(theta)
        yl = y - L * np.sin(theta)
        # Right side
        xr = x + L * np.cos(theta)
        yr = y + L * np.sin(theta)

        # Update quadrotor's position
        quadrotor_L.set_data([xl, xr], [yl, yr])
        
        return quadrotor_L,

    S = states.shape[1]
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=S, interval=20, blit=True)
    # anim.save("MPC_without_noise.mp4", writer='ffmpeg', fps=30, dpi=100)
    anim.save("MPC_with_noise.mp4", writer='ffmpeg', fps=30, dpi=100)
    # anim.save("MPC_2D_path.mp4", writer='ffmpeg', fps=30, dpi=100)
    
    plt.close()
    
    return HTML(anim.to_jshtml())

In [ ]:
# --- Cell 5 ---
def dynamics(x, u):
    x, y, theta, x_dot, y_dot, theta_dot = x[0,0], x[1,0], x[2,0], x[3,0], x[4,0], x[5,0]
    T, tau = u[0,0], u[1,0]
    vx =  jnp.cos(theta) * x_dot + jnp.sin(theta) * y_dot
    vy = -jnp.sin(theta) * x_dot + jnp.cos(theta) * y_dot
    
    fx = -kx * vx * jnp.abs(vx)
    fy = -ky * vy * jnp.abs(vy)
    f_theta = -k_theta * theta_dot
    
    x_ddot = 1/m * (-jnp.sin(theta) * (T + fy) + jnp.cos(theta) * fx)
    y_ddot = 1/m * ( jnp.cos(theta) * (T + fy) + jnp.sin(theta) * fx) - g
    theta_ddot = (tau + f_theta) / J
    
    f = jnp.array([[x_dot, y_dot, theta_dot, x_ddot, y_ddot, theta_ddot]]).T

    return f

# Runge-Kutta 4th Order integration
def rk4(x, u):
    f1 = dynamics(x, u)
    f2 = dynamics(x + f1 * dt/2, u)
    f3 = dynamics(x + f2 * dt/2, u)
    f4 = dynamics(x + f3 * dt, u)
    
    return x + dt/6 * (f1 + 2*f2 + 2*f3 + f4)

# Linearization
def linearize(x_bar, u_bar):
    A = (jacfwd(rk4, argnums=0)(x_bar, u_bar)).reshape(N,N)
    B = (jacfwd(rk4, argnums=1)(x_bar, u_bar)).reshape(N,M)
    
    return A, B

# Cost function
def cost(xs, us, x_bar, u_bar, x_ref, u_ref):
    c = 1/2 * ((xs[-1] - x_bar) - (x_ref - x_bar)).T @ QF @ ((xs[-1] - x_bar) - (x_ref - x_bar))

    for k in range(S-1):
        c += 1/2 * ((xs[k] - x_bar) - (x_ref - x_bar)).T @ Q @ ((xs[k] - x_bar) - (x_ref - x_bar))
        c += 1/2 * ((us[k] - u_bar) - (u_ref - u_bar)).T @ R @ ((us[k] - u_bar) - (u_ref - u_bar))
        
    return c[0][0]

Let's first compute the optimal K, P gains with LQR.

In [ ]:
# --- Cell 6 ---
def lqr(A, B):
    Ks = [np.zeros((M, N))] * (S-1)
    Ps = [np.zeros((N, N))] * S

    # Terminal cost
    Ps[S-1] = QF
    
    # Riccati backward
    for k in range(S-2, -1, -1):
        lhs = (R + B.T @ Ps[k+1] @ B)
        rhs = B.T @ Ps[k+1] @ A
        Ks[k] = np.linalg.solve(lhs, rhs)
        Ps[k] = Q + Ks[k].T @ R @ Ks[k] + (A - B @ Ks[k]).T @ Ps[k+1] @ (A - B @ Ks[k]) # P expression from DP
    
    # The sequence converges to a steady-state solution
    # !Attention! We have BACKWARD recursion and Ps[0], Ks[0] are not matrices in the 1st step, but in last one.
    Kinf = Ks[0]
    Pinf = Ps[0]

    return Kinf, Pinf

In [ ]:
# --- Cell 7 ---
# MPC stuff!
tc = 0.75 # 0.75 sec look ahead (horizon)
Nh = int(tc/dt)
print(f"Nh = {Nh}")

In [ ]:
# --- Cell 8 ---
# Initial state and control
x0 = np.array([[0., 0., 0., 0., 0., 0.]]).T
u0 = np.array([[0. , 0.]]).T

# Reference state and control
# Without noise
# x_ref = np.array([[25., 160., 0., 0., 0., 0.]]).T
# With noise
x_ref = np.array([[20., 90., 0., 0., 0., 0.]]).T

# x_ref = np.array([[16., 16., 0., 0., 0., 0.]]).T
u_ref = np.array([[m*g, 0.]]).T

# Εquilibrium state
x_bar = np.array([[0., 0., 0., 0., 0., 0.]]).T
u_bar = np.array([[m*g, 0.]]).T

# Linearization around the equilibrium state
A, B = linearize(x_bar, u_bar)

# LQR
Kinf, Pinf = lqr(A, B)

Let's build the QP matrices.

In [ ]:
# --- Cell 9 ---
# Cost matrix
H = np.zeros(((Nh-1) * (M+N), (Nh-1) * (M+N)), np.float64)
H[:M, :M] = R      # first block
H[-N:, -N:] = Pinf # last block

for k in range(Nh-2):
    idx_start = M + k * (M+N)
    idx_end   = idx_start + N
    H[idx_start:idx_end, idx_start:idx_end] = Q

    idx_start = (M+N) + k * (M+N)
    idx_end   = idx_start + M
    H[idx_start:idx_end, idx_start:idx_end] = R

# Cost vector
h = np.zeros(((Nh-1) * (M+N),), np.float64)

# Equality Constraints (dynamics)
G = np.zeros(((Nh-1) * N, (Nh-1) * (M+N)), np.float64)
G[:N, :M] = B
G[:N, M:(M+N)] = -np.eye(N)

for k in range(Nh-2):
    idx_start1 = (k+1) * N
    idx_start2 = k * (M+N) + M
    
    G[idx_start1:idx_start1+N, idx_start2:idx_start2+N] = A
    G[idx_start1:idx_start1+N, idx_start2+N:idx_start2+N+M] = B
    G[idx_start1:idx_start1+N, idx_start2+N+M:idx_start2+N+M+N] = -np.eye(N)

d = np.zeros(((Nh-1) * N,), np.float64)
d[:N] = (-A @ (x0 - x_bar)).reshape(-1,)

# Inequality Constraints (thrust/torque limits)
C = np.zeros(((Nh-1) * M, (Nh-1) * (M+N)), np.float64)
l = np.zeros(((Nh-1) * M,), np.float64)
u = np.zeros(((Nh-1) * M,), np.float64)

for k in range(Nh-1):
    C[(k*M):(k*M) + M, k * (M+N):k * (M+N) + M] = np.eye(M)

    l[(k*M):(k*M) + M] = np.array([Tmin - u_bar[0,0], tau_min - u_bar[1,0]]).reshape(-1,)
    u[(k*M):(k*M) + M] = np.array([Tmax - u_bar[0,0], tau_max - u_bar[1,0]]).reshape(-1,)

In [ ]:
# --- Cell 10 ---
# Dimensions of QP problem
qp_dim = H.shape[0]
qp_dim_eq = G.shape[0]
qp_dim_in = C.shape[0]

qp = proxsuite.proxqp.dense.QP(qp_dim, qp_dim_eq, qp_dim_in)

# Initialize the model of the problem to solve
qp.init(H, h, G, d, C, l, u)

# Give an initial solve
qp.solve()

# Solution
prev_sol_x = qp.results.x
prev_sol_eq = qp.results.y
prev_sol_ineq = qp.results.z

# **3. Controlling via Convex MPC**

## 3.1 Convex MPC

Reminder of the **“Convex Model Predictive Control”**:

$$\begin{aligned} \min_{\{x_{1:H},\, u_{1:H-1}\}} \quad & \sum_{k=1}^{H-1} (\frac{1}{2}x_k^\top Q_k x_k + \frac{1}{2}u_k^\top R_k u_k) + \frac{1}{2}x_H^\top P_H x_H \\ \text{s.t.} \quad & x_{k+1} = A x_k + B u_k, \\ & u_k \in \mathcal{U}.\end{aligned}$$

where **$P_H$** is the infinite LQR **P** matrix.

## 3.2 Task
In this part, you are asked to implement a Convex MPC controller for the system. In particular:

1. Write a function in Python called `mpc_controller()` that takes as input the current state of the system and the target state (reference), and outputs the controls/torques to be applied to the system. The function should implement Convex MPC as a Quadratic Programming. In other words, you should use the linearized dynamics.

In [ ]:
# --- Cell 11 ---
def mpc_controller(x, x_bar, u_bar, x_ref, u_ref):
    global prev_sol_x, prev_sol_eq, prev_sol_ineq
    global H, h, G, d, C, l, u
    global A, B, Pinf, Kinf

    # Let's update the matrices
    d[:N] = (- A @ (x - x_bar)).reshape(-1,)
   
    for k in range(Nh-2):
        idx_start = k * (M+N)
        idx_end = k * (M+N) + M
        h[idx_start:idx_end] = (-R @ (u_ref - u_bar)).reshape(-1,)
        
        idx_start = k * (M+N) + M
        idx_end = k * (M+N) + (M+N)
        
        h[idx_start:idx_end] = (-Q @ (x_ref - x_bar)).reshape(-1,)
            
    h[-N:] = (-Pinf @ (x_ref - x_bar)).reshape(-1,)

    # Update qp problem
    qp.update(H, h, G, d, C, l, u)

    # Solve
    qp.solve(prev_sol_x, prev_sol_eq, prev_sol_ineq)

    prev_sol_x = qp.results.x
    prev_sol_eq = qp.results.y
    prev_sol_ineq = qp.results.z

    du = qp.results.x[:M].reshape((M, -1))
    first_u = du + u_bar
    
    return first_u

Let's test the MPC controller!

In [ ]:
# --- Cell 12 ---
us = []
xs = [np.copy(x0)]

x = np.copy(x0)

for k in range(S-1):
    u_new = mpc_controller(x, x_bar, u_bar, x_ref, u_ref)
    us.append(np.copy(u_new))

    x = rk4(x, u_new)
    xs.append(np.copy(x))
    
print(f"Total cost (without noise): {cost(xs, us, x_bar, u_bar, x_ref, u_ref):.2f}")
# print("Wait for simulation...")
# simulation(xs, x0, x_ref, None)

In [ ]:
# --- Cell 13 ---
# Let's plot it!
xs = np.array(xs)
us = np.array(us)

fig = plt.figure(figsize=(15, 5))
fig.suptitle("MPC results without noise ($N_h = 15$) at hover point $(x,y,θ) = (25, 160, 0)$", fontsize=10, fontweight='bold', y=0.95)

ax1 = fig.add_subplot(131)
ax1.plot([n*dt for n in range(len(xs))], xs[:,0], label=r'x')
ax1.plot([n*dt for n in range(len(xs))], xs[:,1], label=r'y')
ax1.plot([n*dt for n in range(len(xs))], xs[:,2], label=r'θ')
ax1.legend()

ax2 = fig.add_subplot(132)
ax2.plot([n*dt for n in range(len(xs))], xs[:,3], label=r'$\dot{x}$')
ax2.plot([n*dt for n in range(len(xs))], xs[:,4], label=r'$\dot{y}$')
ax2.plot([n*dt for n in range(len(xs))], xs[:,5], label=r'$\dot{θ}$')
ax2.legend()

ax3 = fig.add_subplot(133)
ax3.plot([n*dt for n in range(len(us))], us[:,0], label=r'T')
ax3.plot([n*dt for n in range(len(us))], us[:,1], label=r'τ')
ax3.legend()

# fig.savefig('file_name.png', dpi=300, bbox_inches='tight')

plt.show()

2. Choose a horizon length *H* and add noise to the observations (Gaussian with zero mean and diagonal covariance). Compare the LQR controller and the MPC one. Evaluate how far from the linearization point each controller can go.

In [ ]:
# --- Cell 14 ---
cov = np.array([1., 1., 0.01, 0.5, 0.5, 0.01]).reshape(-1, 1)

us_noise = []
xs_noise = [np.copy(x0)]

x = np.copy(x0)
for k in range(S-1):
    noise = np.random.normal(0, cov, (N, 1))
    observation = x + noise
    
    u_new = mpc_controller(observation, x_bar, u_bar, x_ref, u_ref)
    us_noise.append(np.copy(u_new))

    x = rk4(x, u_new)
    xs_noise.append(np.copy(x))
    
print(f"Total cost (with noise): {cost(xs_noise, us_noise, x_bar, u_bar, x_ref, u_ref):.2f}")
# print("Wait for simulation...")
# simulation(xs_noise, x0, x_ref, None)

In [ ]:
# --- Cell 15 ---
# Let's plot it!
xs_noise = np.array(xs_noise)
us_noise = np.array(us_noise)

fig = plt.figure(figsize=(15, 5))
fig.suptitle("MPC results with noise ($N_h = 15$) at hover point $(x,y,θ) = (25, 160, 0)$", fontsize=10, fontweight='bold', y=0.95)

ax1 = fig.add_subplot(131)
ax1.plot([n*dt for n in range(len(xs_noise))], xs_noise[:,0], label=r'x')
ax1.plot([n*dt for n in range(len(xs_noise))], xs_noise[:,1], label=r'y')
ax1.plot([n*dt for n in range(len(xs_noise))], xs_noise[:,2], label=r'θ')
ax1.legend()

ax2 = fig.add_subplot(132)
ax2.plot([n*dt for n in range(len(xs_noise))], xs_noise[:,3], label=r'$\dot{x}$')
ax2.plot([n*dt for n in range(len(xs_noise))], xs_noise[:,4], label=r'$\dot{y}$')
ax2.plot([n*dt for n in range(len(xs_noise))], xs_noise[:,5], label=r'$\dot{θ}$')
ax2.legend()

ax3 = fig.add_subplot(133)
ax3.plot([n*dt for n in range(len(us_noise))], us_noise[:,0], label=r'T')
ax3.plot([n*dt for n in range(len(us_noise))], us_noise[:,1], label=r'τ')
ax3.legend()

# fig.savefig('file_name.png', dpi=300, bbox_inches='tight')

plt.show()

3. Experiment with different horizon lengths. Is it possible to perform a target from any random point and orientation? Can we stabilize at a different point?

4. Can you make the quadrotor follow a 2D path?

In [ ]:
# --- Cell 16 ---
# Let's do a cubic spline

# First calculate coefficients
def calculate_coeffs(xs, xg, T):
    qs = jnp.block([xs[0,0], xs[1,0], xs[2,0]])
    vs = jnp.block([xs[3,0], xs[4,0], xs[5,0]])
    qg = jnp.block([xg[0,0], xg[1,0], xg[2,0]])
    vg = jnp.block([xg[3,0], xg[4,0], xg[5,0]])

    c0 = qs
    c1 = vs
    c2 = ((3 * qg) / (T**2)) - ((3 * qs) / (T**2)) - ((2 * vs) / T) - (vg / T)
    c3 = (-(2 * qg) / (T**3)) + ((2 * qs) / (T**3)) + (vs / (T**2)) + (vg / (T**2))

    return c0, c1, c2, c3

def cubic_spline(t, c0, c1, c2, c3):
    return c3 * (t**3) + c2 * (t**2) + c1 * t + c0

def cubic_spline_dot(t, c0, c1, c2, c3):
    return 3 * c3 * (t**2) + 2 * c2 * t + c1

In [ ]:
# --- Cell 17 ---
def trajectory(knot_points, durations, dt):
    ref_trajectory = []
    total_knot_points = len(knot_points) - 1
    
    for i in range(total_knot_points):
        xs = (knot_points[i]).reshape(6,1)
        xg = (knot_points[i+1]).reshape(6,1)
        T = durations[i]
        
        c0, c1, c2, c3 = calculate_coeffs(xs, xg, T)
        
        steps = int(T/dt)
        for k in range(steps):
            t = k * dt
            q = cubic_spline(t, c0, c1, c2, c3)
            q_dot = cubic_spline_dot(t, c0, c1, c2, c3)
            x = np.block([q, q_dot]) # state
            
            ref_trajectory.append(x)

    last_x = (knot_points[-1]).reshape(6,1)
    final_x = np.block([last_x[0,0], last_x[1,0], 0., 0., 0., 0.])
    
    for _ in range(Nh):
        ref_trajectory.append(final_x)
        
    return np.array(ref_trajectory)

In [ ]:
# --- Cell 18 ---
# Let's fix the MPC controller for trajectory tracking
def mpc_tracking_controller(x, x_bar, u_bar, ref_trajectory, u_ref):
    global prev_sol_x, prev_sol_eq, prev_sol_ineq
    global H, h, G, d, C, l, u
    global A, B, Pinf, Kinf

    d[:N] = (-A @ (x - x_bar)).reshape(-1,)
   
    for k in range(Nh - 1):
        xk = ref_trajectory[k].reshape(-1, 1)
        
        idx_start_u = k * (M+N)
        idx_end_u = k * (M+N) + M
        h[idx_start_u:idx_end_u] = (-R @ (u_ref - u_bar)).reshape(-1,)
        
        idx_start_x = k * (M+N) + M
        idx_end_x = k * (M+N) + (M+N)
        h[idx_start_x:idx_end_x] = (-Q @ (xk - x_bar)).reshape(-1,)
            
    x_final = ref_trajectory[Nh-1].reshape(-1, 1)
    h[-N:] = (-Pinf @ (x_final - x_bar)).reshape(-1,)

    # Update QP problem
    qp.update(H, h, G, d, C, l, u)

    # Solve
    if prev_sol_x is None:
        qp.solve()
    else:
        qp.solve(prev_sol_x, prev_sol_eq, prev_sol_ineq)

    prev_sol_x = qp.results.x
    prev_sol_eq = qp.results.y
    prev_sol_ineq = qp.results.z

    du = qp.results.x[:M].reshape((M, -1))
    first_u = du + u_bar
    
    return first_u

In [ ]:
# --- Cell 19 ---
# Setup trajectory: (0,0) -> (60,0) -> (60,60) -> (0,60) -> (0,0)
knot_points = np.array([[0., 0., 0., 0., 0., 0.], [0., 60., 0., 0., 0., 0.], [60., 60., 0., 0., 0., 0.], [60., 0., 0., 0., 0., 0.], [0., 0., 0., 0., 0., 0.]])
durations = [20, 20, 20, 20]

# Initial state and control
x0 = (knot_points[0]).reshape(6,1)
u0 = np.array([[0. , 0.]]).T

# Reference control
u_ref = np.array([[m*g, 0.]]).T

# Εquilibrium state
x_bar = np.array([[0., 0., 0., 0., 0., 0.]]).T
u_bar = np.array([[m*g, 0.]]).T

# Linearization around the equilibrium state
A, B = linearize(x_bar, u_bar)

# LQR
Kinf, Pinf = lqr(A, B)

In [ ]:
# --- Cell 20 ---
# Cost matrix
H = np.zeros(((Nh-1) * (M+N), (Nh-1) * (M+N)), np.float64)
H[:M, :M] = R      # first block
H[-N:, -N:] = Pinf # last block

for k in range(Nh-2):
    idx_start = M + k * (M+N)
    idx_end   = idx_start + N
    H[idx_start:idx_end, idx_start:idx_end] = Q

    idx_start = (M+N) + k * (M+N)
    idx_end   = idx_start + M
    H[idx_start:idx_end, idx_start:idx_end] = R

# Cost vector
h = np.zeros(((Nh-1) * (M+N),), np.float64)

# Equality Constraints (dynamics)
G = np.zeros(((Nh-1) * N, (Nh-1) * (M+N)), np.float64)
G[:N, :M] = B
G[:N, M:(M+N)] = -np.eye(N)

for k in range(Nh-2):
    idx_start1 = (k+1) * N
    idx_start2 = k * (M+N) + M
    
    G[idx_start1:idx_start1+N, idx_start2:idx_start2+N] = A
    G[idx_start1:idx_start1+N, idx_start2+N:idx_start2+N+M] = B
    G[idx_start1:idx_start1+N, idx_start2+N+M:idx_start2+N+M+N] = -np.eye(N)

d = np.zeros(((Nh-1) * N,), np.float64)
d[:N] = (-A @ (x0 - x_bar)).reshape(-1,)

# Inequality Constraints (thrust/torque limits)
C = np.zeros(((Nh-1) * M, (Nh-1) * (M+N)), np.float64)
l = np.zeros(((Nh-1) * M,), np.float64)
u = np.zeros(((Nh-1) * M,), np.float64)

for k in range(Nh-1):
    C[(k*M):(k*M) + M, k * (M+N):k * (M+N) + M] = np.eye(M)

    l[(k*M):(k*M) + M] = np.array([Tmin - u_bar[0,0], tau_min - u_bar[1,0]]).reshape(-1,)
    u[(k*M):(k*M) + M] = np.array([Tmax - u_bar[0,0], tau_max - u_bar[1,0]]).reshape(-1,)

In [ ]:
# --- Cell 21 ---
# Dimensions of QP problem
qp_dim = H.shape[0]
qp_dim_eq = G.shape[0]
qp_dim_in = C.shape[0]

qp = proxsuite.proxqp.dense.QP(qp_dim, qp_dim_eq, qp_dim_in)

# Initialize the model of the problem to solve
qp.init(H, h, G, d, C, l, u)

# Give an initial solve
qp.solve()

# Solution
prev_sol_x = qp.results.x
prev_sol_eq = qp.results.y
prev_sol_ineq = qp.results.z

In [ ]:
# --- Cell 22 ---
traj = trajectory(knot_points, durations, dt)
total_steps = len(traj) - (Nh)

x0 = np.array([[0., 0., 0., 0., 0., 0.]]).T

us_track = []
xs_track = [np.copy(x0)]

x = np.copy(x0)
for k in range(total_steps):
    ref_window = traj[(k + 1):(k + 1 + Nh)]
    
    u_new = mpc_tracking_controller(x, x_bar, u_bar, ref_window, u_ref)
    us_track.append(np.copy(u_new))

    x = rk4(x, u_new)
    xs_track.append(np.copy(x))

print("Program found the solution!")
# print("Wait for simulation...")
# simulation(xs_track, x0, x0, traj=traj)

In [ ]:
# --- Cell 23 ---
xs_track = np.array(xs_track)
us_track = np.array(us_track)

ref_length = len(xs_track)
ref_xs = traj[:ref_length, 0]
ref_ys = traj[:ref_length, 1]

fig = plt.figure(figsize=(15, 5))
fig.suptitle("2D Path Tracking Results with Convex MPC", fontsize=10, fontweight='bold', y=0.95)

time_axis_x = [n*dt for n in range(len(xs_track))]

ax1 = fig.add_subplot(131)
ax1.plot(time_axis_x, xs_track[:,0], label=r'$x$')
ax1.plot(time_axis_x, xs_track[:,1], label=r'$y$')
ax1.plot(time_axis_x, xs_track[:,2], label=r'$\theta$')
ax1.plot(time_axis_x, ref_xs, 'k--', alpha=0.5, label=r'$x_{ref}$')
ax1.plot(time_axis_x, ref_ys, 'k:', alpha=0.5, label=r'$y_{ref}$')
ax1.legend()

ax2 = fig.add_subplot(132)
ax2.plot(time_axis_x, xs_track[:,3], label=r'$\dot{x}$')
ax2.plot(time_axis_x, xs_track[:,4], label=r'$\dot{y}$')
ax2.plot(time_axis_x, xs_track[:,5], label=r'$\dot{\theta}$')
ax2.legend()

time_axis_u = [n*dt for n in range(len(us_track))]

ax3 = fig.add_subplot(133)
ax3.plot(time_axis_u, us_track[:,0], label=r'$T$')
ax3.plot(time_axis_u, us_track[:,1], label=r'$\tau$')
ax3.legend()

# fig.savefig('file_name.png', dpi=300, bbox_inches='tight')

plt.tight_layout()
plt.show()